In [1]:
"""import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.preprocessing import RobustScaler

# Load dataset
df = pd.read_csv('f1_2019_to_2023_all_drivers_all_data.csv', low_memory=False)

# Convert time columns to seconds
time_columns = ['LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time']
for col in time_columns:
    df[col] = pd.to_timedelta(df[col]).dt.total_seconds()

df['PitOutTime'] = pd.to_datetime(df['PitOutTime'], errors='coerce')
df['PitInTime'] = pd.to_datetime(df['PitInTime'], errors='coerce')

# Drop rows where either PitOutTime or PitInTime is NaT (Not a Time)
df = df.dropna(subset=['PitOutTime', 'PitInTime'])

# Calculate Pit Stop Duration in seconds
df['PitStopDuration'] = (df['PitInTime'] - df['PitOutTime']).dt.total_seconds()

# Check if the new column is in the DataFrame and proceed with IQR analysis
if 'PitStopDuration' in df.columns:
    # Calculate the quartiles and IQR for pit stop times
    Q1 = df['PitStopDuration'].quantile(0.25)
    Q3 = df['PitStopDuration'].quantile(0.75)
    IQR = Q3 - Q1

    # Calculate the bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Check if the mean pit stop time falls within the IQR
    mean_pit_stop_time = 20  # seconds
    if lower_bound <= mean_pit_stop_time <= upper_bound:
        print(f"The mean pit stop time of {mean_pit_stop_time}s is within the IQR ({lower_bound:.2f}s, {upper_bound:.2f}s).")
    else:
        print(f"The mean pit stop time of {mean_pit_stop_time}s is NOT within the IQR ({lower_bound:.2f}s, {upper_bound:.2f}s).")
else:
    print("PitStopDuration column does not exist in the dataset. Please check the dataset.") """


The mean pit stop time of 20s is NOT within the IQR (nans, nans).


/var/folders/_5/533f3cq93vq25vl2f3txpkb40000gn/T/ipykernel_39552/4129554204.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['PitOutTime'] = pd.to_datetime(df['PitOutTime'], errors='coerce')
/var/folders/_5/533f3cq93vq25vl2f3txpkb40000gn/T/ipykernel_39552/4129554204.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['PitInTime'] = pd.to_datetime(df['PitInTime'], errors='coerce')


In [12]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
from xgboost import XGBRegressor
# Load dataset
df = pd.read_csv('f1_2019_to_2023_all_drivers_all_data.csv', low_memory=False)

# Convert time columns to seconds
time_columns = ['LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time']
for col in time_columns:
    df[col] = pd.to_timedelta(df[col]).dt.total_seconds()

# Convert binary columns to integer type
df['Rainfall'] = df['Rainfall'].astype(int)
df['FreshTyre'] = df['FreshTyre'].astype(int)
df['IsAccurate'] = df['IsAccurate'].astype(int)


# Categorize weather condition based on centroid values of Kmeans clustering
def categorize_weather(row):
    if row['Rainfall'] > 0:
        return 'Rainy'
    elif row['AirTemp'] > 28.43213126:
        return 'high'
    elif row['AirTemp'] > 21.31279265:
        return 'medium'
    elif row['AirTemp'] > 12.84901403:
        return 'low'
    else:
        return 'very_low'
df['Weather_Category'] = df.apply(categorize_weather, axis=1)
df = pd.get_dummies(df, columns=['Weather_Category'])


# Create Track temperature category based on the result of Kmeans clustering 
df['TrackTemp_Cat'] = pd.cut(df['TrackTemp'], bins=[0, 18.96764999, 27.87457484, 35.04425766, 41.75142602, 50.51006013, 53.02449646], labels=['VERY_LOW', 'Low', 'Medium', 'Warm', 'High','VERY_High'])
df = pd.get_dummies(df, columns=['TrackTemp_Cat'])


# One-hot encoding
df = pd.get_dummies(df, columns=['Driver', 'Compound', 'Team','TrackStatus','Circuit'])
# Assuming the track status columns are already one-hot encoded as 'TrackStatus_#'
# and that you have a predefined dictionary of track conditions that significantly impact race dynamics
informative_conditions = ['TrackStatus_4.0', 'TrackStatus_5.0', 'TrackStatus_6.0']  # safety car, red flag, virtual safety car

# Create a new column to categorize laps as 'normal', 'informative', or 'noise'
df['outlier_type'] = 'normal'  # default value


# Drop irrelevant columns
columns_to_drop = ['Time', 'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime',
                   'PitOutTime', 'PitInTime', 'LapStartDate', 'Deleted', 'DeletedReason', 'FastF1Generated',
                   'IsPersonalBest', 'Sector3Time','LapStartTime','Sector2Time','Sector1Time']
df.drop(columns=columns_to_drop, inplace=True)


# Update the outlier_type based on track status
for condition in informative_conditions:
    df.loc[df[condition] == 1, 'outlier_type'] = 'informative'
# Handling Outliers
normal_laps = df[df['outlier_type'] == 'normal']
Q1 = normal_laps['LapTime'].quantile(0.25)
Q3 = normal_laps['LapTime'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df.loc[(df['LapTime'] < lower_bound) | (df['LapTime'] > upper_bound), 'outlier_type'] = 'noise'
df = df[df['outlier_type'] != 'noise']



# Convert other categorical data to dummies if not using XGBoost's native handling
df = pd.get_dummies(df, columns=['outlier_type'])


# Flag for rainy conditions
df['IsRainy'] = df['Rainfall'].apply(lambda x: 1 if x > 0 else 0)

# Separate dataframes for dry and wet conditions
df_dry = df[df['IsRainy'] == 0]
df_wet = df[df['IsRainy'] == 1]


def remove_outliers(df, column_name, multiplier=1.5):
    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    return df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]

# Apply standard IQR for dry days
df_dry_filtered = remove_outliers(df_dry, 'LapTime', multiplier=1.5)

# Apply a more lenient IQR for wet days
df_wet_filtered = remove_outliers(df_wet, 'LapTime', multiplier=2.0)


df_combined = pd.concat([df_dry_filtered, df_wet_filtered], ignore_index=True)



# Define features and target
X = df_combined.drop('LapTime', axis=1)
y = df_combined['LapTime']


train_years = [2019,2020,2021,2022]
test_year = 2023
# Split data based on year
X_train = df_combined[df_combined['Year'].isin(train_years)].drop(['LapTime', 'Year'], axis=1)
y_train = df_combined[df_combined['Year'].isin(train_years)]['LapTime']
X_test = df_combined[df_combined['Year'] == test_year].drop(['LapTime', 'Year'], axis=1)
y_test = df_combined[df_combined['Year'] == test_year]['LapTime']




# Drop rows where the target variable is missing in the training set
train_indices = y_train.dropna().index  # Indices of rows where y_train is not NaN
X_train = X_train.loc[train_indices]
y_train = y_train.dropna()  # Drop missing values in y_train

numeric_features = ['Humidity', 'Pressure', 'WindDirection', 'WindSpeed','TrackTemp','AirTemp','SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST']

X_train[numeric_features] = X_train[numeric_features].fillna(method='ffill')

# Forward fill missing values in the test set
X_test[numeric_features] = X_test[numeric_features].fillna(method='ffill')

# Scale the test set using the same scaler fitted on the training set



scaler = RobustScaler()
X_train[numeric_features] = scaler.fit_transform(X_train[numeric_features])
X_test[numeric_features] = scaler.transform(X_test[numeric_features])



/var/folders/_5/533f3cq93vq25vl2f3txpkb40000gn/T/ipykernel_39632/1278574372.py:131: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_train[numeric_features] = X_train[numeric_features].fillna(method='ffill')
/var/folders/_5/533f3cq93vq25vl2f3txpkb40000gn/T/ipykernel_39632/1278574372.py:134: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_test[numeric_features] = X_test[numeric_features].fillna(method='ffill')


In [13]:
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
from xgboost import XGBRegressor
import matplotlib.pyplot as plt


##BASE XG BOOST

#Grid value :(n_estimators=700, learning_rate=0.1,random_state=42,n_jobs=-1,max_depth=7)

# Train the model with class weight adjustment
xgb_model = XGBRegressor(random_state=42,reg_alpha=10,  # L1 regularization
                         reg_lambda=1)
xgb_model.fit(X_train, y_train)

# Predictions and evaluation
predictions = xgb_model.predict(X_test)

mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
print(f"Baseline RMSE for Combined df: {rmse}")

from sklearn.model_selection import cross_val_score

# Evaluate with K-Fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42,)
scores = cross_val_score(xgb_model, X_train, y_train, cv=kfold, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-scores)  # Convert MSE to RMSE
print("Cross-validated RMSE scores:", rmse_scores)
print("Mean RMSE:", np.mean(rmse_scores))


Baseline RMSE for Combined df: 5.220132276727481
Cross-validated RMSE scores: [1.79234554 1.71571545 1.76707137 1.73264581 1.80629487]
Mean RMSE: 1.762814610166935


In [14]:

#4. XG boost with Random Search

import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import make_scorer, mean_squared_error
kf = KFold(n_splits=5, shuffle=True, random_state=42)
xgb_model = xgb.XGBRegressor(random_state=42)
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800],
    'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
    'max_depth': [3, 5, 7, 9, 11],
    'reg_alpha': [0.1, 1, 10, 100],
    'reg_lambda': [0.1, 1, 10, 100],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0]
}
neg_rmse_scorer = make_scorer(mean_squared_error, greater_is_better=False, squared=True)
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist, n_iter=100, cv=kf,
                                   scoring={'RMSE': neg_rmse_scorer}, refit='RMSE', random_state=42, verbose=3, n_jobs=-1)
random_search.fit(X, y)  # X and y must be your preprocessed datasets
best_model = random_search.best_estimator_
best_params = random_search.best_params_
best_rmse = (-random_search.best_score_) ** 0.5  # Converting MSE to RMSE

print("Best model parameters:", best_params)
print(f"Optimized CV RMSE: {best_rmse:.3f}")


Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 4/5] END colsample_bytree=0.8, learning_rate=0.05, max_depth=3, n_estimators=700, reg_alpha=100, reg_lambda=0.1, subsample=1.0; RMSE: (test=-5.703) total time=   8.6s
[CV 3/5] END colsample_bytree=0.9, learning_rate=0.2, max_depth=11, n_estimators=500, reg_alpha=0.1, reg_lambda=100, subsample=0.9; RMSE: (test=-2.034) total time=  20.1s
[CV 5/5] END colsample_bytree=0.7, learning_rate=0.2, max_depth=7, n_estimators=100, reg_alpha=1, reg_lambda=1, subsample=0.7; RMSE: (test=-2.763) total time=   2.5s
[CV 2/5] END colsample_bytree=0.7, learning_rate=0.1, max_depth=7, n_estimators=200, reg_alpha=100, reg_lambda=10, subsample=0.8; RMSE: (test=-3.297) total time=   4.9s
[CV 2/5] END colsample_bytree=0.9, learning_rate=0.2, max_depth=7, n_estimators=600, reg_alpha=0.1, reg_lambda=10, subsample=0.9; RMSE: (test=-1.919) total time=  11.9s
[CV 3/5] END colsample_bytree=0.8, learning_rate=0.15, max_depth=7, n_estimators=800, reg_a

/opt/anaconda3/envs/COMS30035_labs/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV 5/5] END colsample_bytree=0.9, learning_rate=0.2, max_depth=9, n_estimators=700, reg_alpha=100, reg_lambda=10, subsample=1.0; RMSE: (test=-2.328) total time=  12.5s
[CV 3/5] END colsample_bytree=0.8, learning_rate=0.1, max_depth=9, n_estimators=500, reg_alpha=100, reg_lambda=10, subsample=0.8; RMSE: (test=-2.294) total time=  17.0s
[CV 1/5] END colsample_bytree=0.9, learning_rate=0.01, max_depth=9, n_estimators=600, reg_alpha=1, reg_lambda=10, subsample=0.7; RMSE: (test=-3.062) total time=  16.3s
[CV 1/5] END colsample_bytree=0.9, learning_rate=0.01, max_depth=3, n_estimators=100, reg_alpha=100, reg_lambda=10, subsample=0.9; RMSE: (test=-78.901) total time=   2.7s
[CV 1/5] END colsample_bytree=0.8, learning_rate=0.1, max_depth=5, n_estimators=800, reg_alpha=100, reg_lambda=100, subsample=0.9; RMSE: (test=-2.772) total time=  10.8s
[CV 1/5] END colsample_bytree=0.7, learning_rate=0.05, max_depth=3, n_estimators=300, reg_alpha=100, reg_lambda=10, subsample=1.0; RMSE: (test=-9.920) to

In [3]:

#4. XG boost with Random Search

import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import make_scorer, mean_squared_error
kf = KFold(n_splits=5, shuffle=True, random_state=42)
xgb_model = xgb.XGBRegressor(random_state=42)
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800],
    'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
    'max_depth': [3, 5, 7, 9, 11],
    'reg_alpha': [0.1, 1, 10, 100],
    'reg_lambda': [0.1, 1, 10, 100],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0]
}
neg_rmse_scorer = make_scorer(mean_squared_error, greater_is_better=False, squared=True)
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist, n_iter=100, cv=kf,
                                   scoring={'RMSE': neg_rmse_scorer}, refit='RMSE', random_state=42, verbose=3, n_jobs=-1)
random_search.fit(X, y)  # X and y must be your preprocessed datasets
best_model = random_search.best_estimator_
best_params = random_search.best_params_
best_rmse = (-random_search.best_score_) ** 0.5  # Converting MSE to RMSE

print("Best model parameters:", best_params)
print(f"Optimized CV RMSE: {best_rmse:.3f}")


Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 2/5] END colsample_bytree=0.7, learning_rate=0.01, max_depth=5, n_estimators=600, reg_alpha=1, reg_lambda=100, subsample=0.7; RMSE: (test=-10.610) total time=   9.3s
[CV 1/5] END colsample_bytree=1.0, learning_rate=0.1, max_depth=7, n_estimators=300, reg_alpha=100, reg_lambda=100, subsample=0.7; RMSE: (test=-3.170) total time=   8.7s
[CV 4/5] END colsample_bytree=1.0, learning_rate=0.1, max_depth=7, n_estimators=300, reg_alpha=100, reg_lambda=100, subsample=0.7; RMSE: (test=-3.689) total time=   9.2s
[CV 4/5] END colsample_bytree=0.7, learning_rate=0.2, max_depth=7, n_estimators=100, reg_alpha=1, reg_lambda=1, subsample=0.7; RMSE: (test=-3.325) total time=   3.0s


/opt/anaconda3/envs/COMS30035_labs/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV 1/5] END colsample_bytree=0.7, learning_rate=0.01, max_depth=5, n_estimators=600, reg_alpha=1, reg_lambda=100, subsample=0.7; RMSE: (test=-10.134) total time=   9.1s
[CV 5/5] END colsample_bytree=0.9, learning_rate=0.2, max_depth=11, n_estimators=500, reg_alpha=0.1, reg_lambda=100, subsample=0.9; RMSE: (test=-2.224) total time=  21.5s
[CV 3/5] END colsample_bytree=0.7, learning_rate=0.01, max_depth=5, n_estimators=600, reg_alpha=1, reg_lambda=100, subsample=0.7; RMSE: (test=-11.508) total time=   8.8s
[CV 4/5] END colsample_bytree=0.9, learning_rate=0.2, max_depth=11, n_estimators=500, reg_alpha=0.1, reg_lambda=100, subsample=0.9; RMSE: (test=-2.600) total time=  19.0s
[CV 5/5] END colsample_bytree=0.7, learning_rate=0.2, max_depth=7, n_estimators=100, reg_alpha=1, reg_lambda=1, subsample=0.7; RMSE: (test=-3.086) total time=   2.8s
[CV 5/5] END colsample_bytree=0.8, learning_rate=0.05, max_depth=3, n_estimators=700, reg_alpha=100, reg_lambda=0.1, subsample=1.0; RMSE: (test=-6.226) 